In [1]:
import numpy as np
import random

In [22]:
# Define the alphabet
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .'

# Convert text to binary sequence
def text_to_binary(text):
    binary_seq = ''.join(format(alphabet.index(c), '06b') for c in text)
    return np.array([int(b) for b in binary_seq])

# Convert binary sequence to codeword with n samples
def binary_to_codeword(binary_seq, n):
    codeword = np.zeros(len(binary_seq)*n)
    for i in range(len(binary_seq)):
        codeword[i*n:(i+1)*n] = (-1)**(int(binary_seq[i]))  # Map 0 to 1 and 1 to -1
    return codeword

In [57]:
print(text_to_binary('H'))
print(len(binary_to_codeword(text_to_binary('H'), 32)))

[1 0 0 0 0 1]
192


In [24]:
print(binary_to_codeword('000000', 32))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [105]:
# Convert binary sequence back to text
def binary_to_text(binary_seq):
    binary_str = ''.join(map(str, binary_seq))
    text = ''.join(alphabet[int(binary_str[i:i+6], 2)] for i in range(0, len(binary_str), 6))
    return text

# Simulate the channel
def channel(x):
    n = x.size
    B = random.choice([0, 1])
    sigma_sq = 25
    Z = np.random.normal(0, np.sqrt(sigma_sq), 2 * n)
    X = np.zeros(2 * n)
    if B == 1:
        X[:n] = x
    else:
        X[n:2*n] = x
    Y = X + Z
    Y = np.reshape(Y, (-1))
    return Y

CODEBOOK = {'0': 5*np.ones(32), '1': (-5)*np.ones(32)}

# Save codebook to a file
def save_codebook_to_file(codebook, file_path):
    np.save(file_path, codebook)

# Load codebook from a file
def load_codebook_from_file(file_path):
    return np.load(file_path)



# Transmitter
message = "Hello World Hello World Hillo World 1.1."
print("Original Message:", message)
binary_seq = text_to_binary(message)
codeword = binary_to_codeword(binary_seq, 32)
    #np.savetxt('input_signal.txt', codeword)

    # Channel
    #transmitted_signal = np.loadtxt('input_signal.txt')
transmitted_signal = codeword

received_signal = channel(transmitted_signal)
    #np.savetxt('received_signal.txt', received_signal)

    


Original Message: Hello World Hello World Hillo World 1.1.


In [106]:
# Receiver
    #received_signal = np.loadtxt('received_signal.txt')
message_code = []
for i in range(40):
    message_code.append(received_signal[i*64*6:(i+1)*64*6])
message_bin = []
for i in range(40):
    char_bin = ''
    for j in range(6):
        y1 = message_code[i][j*64:j*64+32]
        y2 = message_code[i][j*64+32:(j+1)*64]
        i1 = np.argmin([np.linalg.norm(y1 - c) for c in CODEBOOK.values()])
        i2 = np.argmin([np.linalg.norm(y2 - c) for c in CODEBOOK.values()])
        d1 = np.linalg.norm(y1 - CODEBOOK['0']) ** 2 / 25 + np.linalg.norm(y2) ** 2 / 25
        d2 = np.linalg.norm(y2 - CODEBOOK['1']) ** 2 / 25 + np.linalg.norm(y1) ** 2 / 25
        if d1 < d2:
            char_bin += str(i1)
        else:
            char_bin += str(i2)
        
    message_bin.append(char_bin)


for i in range(40):
    print(binary_to_text(message_bin[i]), end='')


IzDRBpIlxJRNklwJlo NiOnLCg6NPmmvdk7X1wJS

In [97]:
print(CODEBOOK['0'])

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5.]


In [ ]:
if __name__ == '__main__':
    main()